In [1]:
import sys
import os
from dotenv import load_dotenv
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt

# Load environment variables

In [26]:
load_dotenv()
user = os.environ['PG_USER']
password = os.environ['PG_PASSWORD']
host = os.environ['PG_HOST']
port = os.environ['PG_PORT']
database = 'postgres'

# Connect to Database

In [30]:
def connect():
    conn = None
    try:
        print('Connecting..')
        conn = psycopg2.connect(
              host=host,
              database=database,
              user=user,
              password=password
        )
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        sys.exit(1)   
        
    print("All good, Connection successful!")
    return conn

In [28]:
def list_tables(conn):
    cursor = conn.cursor()
    try:
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public' ORDER BY table_name")
        tables = cursor.fetchall()
        cursor.close()
        return tables
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return None



In [29]:
# Opening the connection
conn = connect()

# Check if the connection is established
if conn:
    print("Connection to PostgreSQL successful!")
    
    # Get a list of existing tables in the database
    tables = list_tables(conn)
    if tables:
        print("Existing tables in the 'public' schema:")
        for table in tables:
            print(table[0])
    else:
        print("Failed to retrieve table list.")
else:
    print("Failed to connect to PostgreSQL.")

# Closing the connection
if conn is not None:
    conn.close()
    print("Connection closed.")

Connecting..
All good, Connection successful!
Connection to PostgreSQL successful!
Existing tables in the 'public' schema:
banksaddata
googlestorereview
subscriberdata
Connection closed.


In [31]:
def sql_to_dataframe(conn, query):
    """Import data from a PostgreSQL database using a SELECT query"""
    cursor = conn.cursor()   
    try:
        cursor.execute(query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    # The execute returns a list of tuples:   
    tuples_list = cursor.fetchall()   
    
    # Now we need to transform the list into a pandas DataFrame:   
    df = pd.DataFrame(tuples_list, columns=[col[0] for col in cursor.description])
    cursor.close()   
    return df

In [35]:
# Create query
query = """ SELECT * FROM public.banksaddata """

# Opening the connection
conn = connect()

# Loading our dataframe
tikvahBankData = sql_to_dataframe(conn, query)

# Closing the connection
conn.close()


Connecting..
All good, Connection successful!


In [36]:
tikvahBankData.sample(10)

,id,channel_username,message_id,message_link,views,message_content,date,image_html,banks,time
8,9,tikvahethiopia,81123,https://t.me/tikvahethiopia/81123,233514,#Bank_of_Abyssinia<br><br>አቢሲንያ ባንክ ለዕድሮች ባዘጋጀ...,"Aug 29, 2023 at 21:02","width: 800px; background-image: url(""https://c...",BOA,2023-08-29
9,10,tikvahethiopia,76089,https://t.me/tikvahethiopia/76089,129550,አፖሎ የተሰኘው ልዩ የዲጂታል መተግበሪያ ለተጠቃሚ ዝግጁ መሆኑን ስንገልጽ...,"Jan 26, 2023 at 09:49","width: 800px; background-image: url(""https://c...",BOA,2023-01-26
4,5,tikvahethiopia,81338,https://t.me/tikvahethiopia/81338,221056,#Bank_of_Abyssinia<br><br>ዕቁብ ልዩ የቁጠባ ሂሳብ!<br>...,"Sep 8, 2023 at 10:50","width: 800px; background-image: url(""https://c...",BOA,2023-08-09
7,8,tikvahethiopia,81123,https://t.me/tikvahethiopia/81123,233523,#Bank_of_Abyssinia<br><br>አቢሲንያ ባንክ ለዕድሮች ባዘጋጀ...,"Aug 29, 2023 at 21:02","width: 800px; background-image: url(""https://c...",BOA,2023-08-29
2,3,tikvahethiopia,81606,https://t.me/tikvahethiopia/81606,260029,#Bank_of_Abyssinia<br><br>የአቢሲንያ ባንክን የተለያዩ አገ...,"Sep 19, 2023 at 18:53","width: 800px; background-image: url(""https://c...",BOA,2023-09-19
5,6,tikvahethiopia,81252,https://t.me/tikvahethiopia/81252,257632,#Bank_of_Abyssinia<br><br>ለመጪዎቹ የአዲስ አመት፣ የመስቀ...,"Sep 5, 2023 at 13:19","width: 800px; background-image: url(""https://c...",BOA,2023-05-09
3,4,tikvahethiopia,81502,https://t.me/tikvahethiopia/81502,271137,#Bank_of_Abyssinia<br><br>የአቢሲንያ ባንክን የተለያዩ አገ...,"Sep 15, 2023 at 15:56","width: 800px; background-image: url(""https://c...",BOA,2023-09-15
0,1,tikvahethiopia,87858,https://t.me/tikvahethiopia/87858,212588,#Abyssinia_Bank<br><br>ዕሴቶቻችሁንና የሸሪዓን መርሆች በማክ...,May 24 at 11:54,"width: 800px; background-image: url(""https://c...",BOA,2024-05-24
6,7,tikvahethiopia,81165,https://t.me/tikvahethiopia/81165,260547,#Bank_of_Abyssinia<br><br>አቢሲንያ ባንክ ለዕድሮች ባዘጋጀ...,"Sep 1, 2023 at 10:20","width: 800px; background-image: url(""https://c...",BOA,2023-01-09
1,2,tikvahethiopia,81734,https://t.me/tikvahethiopia/81734,241795,#Bank_of_Abyssinia<br><br>የአቢሲንያ ባንክን የተለያዩ አገ...,"Sep 26, 2023 at 10:18","width: 800px; background-image: url(""https://c...",BOA,2023-09-26


In [38]:
# Create query
query = """ SELECT * FROM public.googlestorereview """

# Opening the connection
conn = connect()

# Loading our dataframe
googleStoreReviewData = sql_to_dataframe(conn, query)

# Closing the connection
conn.close()


Connecting..
All good, Connection successful!


In [39]:
googleStoreReviewData.sample(10)

,reviewid,username,userimage,content,score,thumbsupcount,reviewcreatedversion,createdat,replycontent,repliedat,appversion,bank
3,622762e8-14b9-42eb-b807-8960278d8b34,Hermon Z,https://play-lh.googleusercontent.com/a-/ALV-U...,What is that disable developer option I have n...,3,0,24.05.07,2024-06-22 07:03:55,None,None,24.05.07,BOA
6,9340824c-f839-4afe-81af-cd398088f470,Ahadu tesfaye,https://play-lh.googleusercontent.com/a-/ALV-U...,Very poor,1,0,24.05.07,2024-06-21 17:53:34,None,None,24.05.07,BOA
1,58ccb5e9-0cf7-413c-8bb4-6515d4863bc1,Amenti Tesgera,https://play-lh.googleusercontent.com/a/ACg8oc...,🥰🥰🥰🥰🥰,5,0,24.05.07,2024-06-23 11:39:10,None,None,24.05.07,BOA
7,68fe0a10-cfc1-4bdd-b447-271fea684980,Adane Kelelew,https://play-lh.googleusercontent.com/a/ACg8oc...,Excellent,5,0,None,2024-06-21 13:55:00,None,None,None,BOA
0,1c7fe538-d19f-40ac-9b8b-3e3458f8a26e,Ibsaa Carcar,https://play-lh.googleusercontent.com/a-/ALV-U...,Very Very nice 👌 👍,5,0,None,2024-06-23 20:37:43,None,None,None,BOA
8,322c12e9-1e9a-4609-b4c0-3ef5f78ff589,Yonas A,https://play-lh.googleusercontent.com/a-/ALV-U...,Playstore need to have some option to give 0 s...,1,20,24.05.07,2024-06-21 10:17:07,None,None,24.05.07,BOA
2,f77c9ee3-07b7-4203-9aaa-f019d35abaa6,Chernet Bekele,https://play-lh.googleusercontent.com/a/ACg8oc...,Cool,4,1,None,2024-06-22 14:02:36,None,None,None,BOA
5,fe54afea-6f7b-4fee-bc8a-4af38050cc54,Mohammed Kasim,https://play-lh.googleusercontent.com/a-/ALV-U...,All,5,0,24.05.07,2024-06-21 20:38:28,None,None,24.05.07,BOA
4,9756a397-2464-40d6-8eab-d269c119aa9e,Davinci Tube,https://play-lh.googleusercontent.com/a-/ALV-U...,Please fix the app i doesn't belongs to Abbssi...,1,0,24.05.07,2024-06-22 02:01:58,None,None,24.05.07,BOA
9,42e6b999-2d95-4374-bf40-93c60d08c58f,Abdurezak Awol,https://play-lh.googleusercontent.com/a/ACg8oc...,It's good when i've downloaded at first. But n...,2,0,24.05.07,2024-06-20 20:29:06,None,None,24.05.07,BOA


In [40]:
# Create query
query = """ SELECT * FROM public.subscriberdata """

# Opening the connection
conn = connect()

# Loading our dataframe
subscriberData = sql_to_dataframe(conn, query)

# Closing the connection
conn.close()


Connecting..
All good, Connection successful!


In [41]:
subscriberData.sample(10)

,time,subscriber_count,date
151,06:00,0,2024-05-19
80,07:00,18,2024-05-22
22,19:00,1,2024-05-24
23,18:00,49,2024-05-24
60,04:00,5,2024-05-23
142,15:00,10,2024-05-19
188,17:00,40,2024-05-17
219,10:00,3,2024-05-16
31,10:00,1,2024-05-24
143,14:00,6,2024-05-19
